This notebook reads the intersection map information and makes the information transferable into geopandas.

In [1]:
import os 
import glob
import pandas as pd
import numpy as np
from rapidfuzz import process
import ast

In [2]:
intersections = pd.read_csv('intersections.csv')

In [3]:
intersections.head()

,_id,INTERSECTION_ID,DATE_EFFECTIVE,DATE_EXPIRY,ELEVATION_ID,INTERSECTION_DESC,CLASSIFICATION,CLASSIFICATION_DESC,NUMBER_OF_ELEVATIONS,ELEVATION_FEATURE_CODE,...,ELEVATION_LEVEL,ELEVATION,ELEVATION_UNIT,HEIGHT_RESTRICTION,HEIGHT_RESTRICTION_UNIT,STATE,TRANS_ID_CREATE,TRANS_ID_EXPIRE,OBJECTID,geometry
0,1,13470264,NaN,NaN,13,Robindale Ave / Rimilton Ave,MNRSL,Minor-Single Level,1,501300.0,...,0,NaN,NaN,NaN,NaN,8,200000,-1,1,"{'type': 'MultiPoint', 'coordinates': [[-79.53..."
1,2,13470193,NaN,NaN,4718,Bellman Ave / Valermo Dr,MNRSL,Minor-Single Level,1,501300.0,...,0,NaN,NaN,NaN,NaN,8,200000,-1,4,"{'type': 'MultiPoint', 'coordinates': [[-79.53..."
2,3,13470188,NaN,NaN,32728,Rimilton Ave / Valermo Dr,SEUSL,Pseudo Intersection-Single Level,1,509200.0,...,0,NaN,NaN,NaN,NaN,8,200000,-1,5,"{'type': 'MultiPoint', 'coordinates': [[-79.53..."
3,4,13470203,NaN,NaN,21669,Valermo Dr / Goa Crt,MNRSL,Minor-Single Level,1,501300.0,...,0,NaN,NaN,NaN,NaN,8,200000,-1,7,"{'type': 'MultiPoint', 'coordinates': [[-79.53..."
4,5,13470228,NaN,NaN,36820,Valermo Dr / Thirtieth St,MNRSL,Minor-Single Level,1,501300.0,...,0,NaN,NaN,NaN,NaN,8,200000,-1,9,"{'type': 'MultiPoint', 'coordinates': [[-79.53..."


In [4]:
columns_to_keep = ['INTERSECTION_DESC', 'CLASSIFICATION_DESC', 'geometry']

# Drop columns not listed in columns_to_keep
intersections = intersections[columns_to_keep]

In [5]:
intersections.head()

,INTERSECTION_DESC,CLASSIFICATION_DESC,geometry
0,Robindale Ave / Rimilton Ave,Minor-Single Level,"{'type': 'MultiPoint', 'coordinates': [[-79.53..."
1,Bellman Ave / Valermo Dr,Minor-Single Level,"{'type': 'MultiPoint', 'coordinates': [[-79.53..."
2,Rimilton Ave / Valermo Dr,Pseudo Intersection-Single Level,"{'type': 'MultiPoint', 'coordinates': [[-79.53..."
3,Valermo Dr / Goa Crt,Minor-Single Level,"{'type': 'MultiPoint', 'coordinates': [[-79.53..."
4,Valermo Dr / Thirtieth St,Minor-Single Level,"{'type': 'MultiPoint', 'coordinates': [[-79.53..."


Extract the latitude and longitude from the geometry column

In [6]:
intersections['geometry'] = intersections['geometry'].apply(ast.literal_eval)

# Extracting latitude and longitude from 'geometry'
intersections['lat'] = intersections['geometry'].apply(lambda x: x['coordinates'][0][1])
intersections['lon'] = intersections['geometry'].apply(lambda x: x['coordinates'][0][0])

In [7]:
intersections.head()

,INTERSECTION_DESC,CLASSIFICATION_DESC,geometry,lat,lon
0,Robindale Ave / Rimilton Ave,Minor-Single Level,"{'type': 'MultiPoint', 'coordinates': [[-79.53...",43.607243,-79.531070
1,Bellman Ave / Valermo Dr,Minor-Single Level,"{'type': 'MultiPoint', 'coordinates': [[-79.53...",43.609600,-79.531373
2,Rimilton Ave / Valermo Dr,Pseudo Intersection-Single Level,"{'type': 'MultiPoint', 'coordinates': [[-79.53...",43.609829,-79.530118
3,Valermo Dr / Goa Crt,Minor-Single Level,"{'type': 'MultiPoint', 'coordinates': [[-79.53...",43.609190,-79.533175
4,Valermo Dr / Thirtieth St,Minor-Single Level,"{'type': 'MultiPoint', 'coordinates': [[-79.53...",43.608639,-79.535593


Clean up intersection names to match the format of delay location names

In [8]:
intersections['INTERSECTION_DESC'] = intersections['INTERSECTION_DESC'].str.lower()  # Step 1
intersections['INTERSECTION_DESC'] = intersections['INTERSECTION_DESC'].str.replace('/', '&')  # Step 3
intersections['INTERSECTION_DESC'] = intersections['INTERSECTION_DESC'].str.replace(' ', '')
intersections['INTERSECTION_DESC'] = intersections['INTERSECTION_DESC'].apply(lambda x: '&'.join(sorted(str(x).split('&'))) if pd.notna(x) else x)

In [9]:
intersections.head()

,INTERSECTION_DESC,CLASSIFICATION_DESC,geometry,lat,lon
0,rimiltonave&robindaleave,Minor-Single Level,"{'type': 'MultiPoint', 'coordinates': [[-79.53...",43.607243,-79.531070
1,bellmanave&valermodr,Minor-Single Level,"{'type': 'MultiPoint', 'coordinates': [[-79.53...",43.609600,-79.531373
2,rimiltonave&valermodr,Pseudo Intersection-Single Level,"{'type': 'MultiPoint', 'coordinates': [[-79.53...",43.609829,-79.530118
3,goacrt&valermodr,Minor-Single Level,"{'type': 'MultiPoint', 'coordinates': [[-79.53...",43.609190,-79.533175
4,thirtiethst&valermodr,Minor-Single Level,"{'type': 'MultiPoint', 'coordinates': [[-79.53...",43.608639,-79.535593


In [11]:
intersections = intersections.dropna()
intersections = intersections.drop_duplicates(subset=['INTERSECTION_DESC'])

Save as a csv for use in the next section

In [12]:
intersections.to_csv('inter_coord.csv')